In [ ]:
import pandas as pd
import requests
from io import StringIO
import ipywidgets as widgets
from IPython.display import display, clear_output
from tqdm.notebook import tqdm
import yfinance as yf
import time
import warnings

In [ ]:
def group_selector():
    """
    Dropdown to select stock group (ALL, NASDAQ, NYSE, SP500),
    filters tickers to groups (SP500, NASDAQ listed, NYSE listed),
    checks availability on Yahoo Finance, and creates CSV with Symbol + Security Name.
    """

    # Dropdown menu
    options = ["ALL", "NASDAQ", "NYSE", "SP500"]
    result = {'stock_group': None}

    dropdown = widgets.Dropdown(
        options=options,
        value=None,
        description='Group of stocks:',
        style={'description_width': '100px', 'font_weight': 'bold', 'font_size': '16px'},
        layout=widgets.Layout(width='300px', height='40px', margin='10px 0 0 0')
    )

    # Confirmation button
    button = widgets.Button(
        description = "Choose group",
        button_style='success',
        layout=widgets.Layout(width='200px', height='40px', margin='10px 0 10px 0'),
        style={'font_weight': 'bold', 'font_size': '16px'}
    )
    
    output = widgets.Output()
    symbols_output = widgets.Output()

    def on_button_clicked(b):
        with output:
            output.clear_output()
            
            if dropdown.value is None:
                print(f"\U0000274C Please select a stock group.")
                return
            
            choice = dropdown.value
            result['stock_group'] = choice
            print(f"\U00002705 Selected group of stocks: {choice}")
            
            # NASDAQ/NYSE data
            nasdaq_url = "https://www.nasdaqtrader.com/dynamic/SymDir/nasdaqlisted.txt"
            other_url  = "https://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt"

            nasdaq = pd.read_csv(nasdaq_url, sep="|", on_bad_lines="skip")
            other  = pd.read_csv(other_url, sep="|", on_bad_lines="skip")
            nyse = other[other["Exchange"] == "N"]

            # Filter common stocks only
            exclude_keywords = ["Preferred", "Warrant", "Right", "Unit", "Depositary"]

            # NASDAQ
            nasdaq_filtered = nasdaq[
                nasdaq["Symbol"].str.fullmatch(r"[A-Z]+", na=False) &
                (nasdaq['ETF'] == 'N') &
                (nasdaq['NextShares'] == 'N') &
                (nasdaq['Test Issue'] == 'N') &
                (nasdaq['Market Category'].isin(['Q', 'G', 'S'])) &
                (~nasdaq['Security Name'].str.contains('|'.join(exclude_keywords), case=False, na=False))
            ]

            # NYSE
            nyse_filtered = nyse[
                nyse["ACT Symbol"].str.fullmatch(r"[A-Z]+", na=False) &
                (nyse['ETF'] == 'N') &
                (nyse['Test Issue'] == 'N') &
                (~nyse['Security Name'].str.contains('|'.join(exclude_keywords), case=False, na=False))
            ]

            nasdaq_table = nasdaq_filtered[["Symbol", "Security Name"]]
            nyse_table = nyse_filtered.rename(columns={"ACT Symbol": "Symbol"})[["Symbol", "Security Name"]]

            # S&P500 data
            sp500_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
            response = requests.get(sp500_url, headers=headers)
            response.raise_for_status()
            tables = pd.read_html(StringIO(response.text))
            sp500 = tables[0]
            sp500_table = sp500[['Symbol', 'Security']].copy()
            sp500_table = sp500_table.rename(columns={'Security': 'Security Name'})

            # Ticker lists
            sp500_tickers = sp500_table['Symbol'].tolist()
            nasdaq_tickers = nasdaq_table['Symbol'].tolist()
            nyse_tickers = nyse_table['Symbol'].tolist()

            # Define tickers to check
            if choice == "SP500":
                tickers_to_check = sp500_tickers.copy()
            elif choice == "NASDAQ":
                tickers_to_check = nasdaq_tickers.copy()
            elif choice == "NYSE":
                tickers_to_check = nyse_tickers.copy()
            elif choice == "ALL":
                tickers_to_check = list(set(sp500_tickers + nasdaq_tickers + nyse_tickers))
            else:
                print(f"\U0000274C Unknown choice")
                return

            # Yahoo Finance availability control
            tickers_valid = []
            batch_size = 50
            
            for i in tqdm(range(0, len(tickers_to_check), batch_size), leave=False, disable=True):
                batch = tickers_to_check[i:i + batch_size]
                try:
                    with warnings.catch_warnings():
                        warnings.simplefilter("ignore")
                        data = yf.download(batch, period="1d", group_by='ticker', threads=True, progress=False)

                    if isinstance(data.columns, pd.MultiIndex):
                        for t in batch:
                            if t in data.columns.levels[0] and not data[t].dropna(how='all').empty:
                                tickers_valid.append(t)
                    else:
                        t0 = batch[0]
                        if not data.dropna(how='all').empty:
                            tickers_valid.append(t0)

                except (yf.shared._exceptions.YFRateLimitError,
                        yf.shared._exceptions.YFPricesMissingError,
                        TimeoutError, ConnectionError):
                    continue

            tickers_valid_set = set(tickers_valid)

            # Filter available Symbols based on the group choice
            if choice == 'SP500':
                df = sp500_table[sp500_table['Symbol'].isin(tickers_valid_set)].set_index('Symbol')
            elif choice == 'NASDAQ':
                df = nasdaq_table[nasdaq_table['Symbol'].isin(tickers_valid_set)].set_index('Symbol')
            elif choice == 'NYSE':
                df = nyse_table[nyse_table['Symbol'].isin(tickers_valid_set)].set_index('Symbol')
            elif choice == 'ALL':
                combined = pd.concat([
                    sp500_table[sp500_table['Symbol'].isin(tickers_valid_set)],
                    nasdaq_table[nasdaq_table['Symbol'].isin(tickers_valid_set)],
                    nyse_table[nyse_table['Symbol'].isin(tickers_valid_set)]
                ], ignore_index=True)
                df = combined.drop_duplicates(subset='Symbol').set_index('Symbol')

            # Create CSV
            filename = f"{choice}_tickers.csv"
            df.to_csv(filename)
            clear_output(wait=True)
            print(f"\U00002705 CSV with the stocks available for the selected group has been created in your folder: {filename}\n \n                       \U0001F4CA Data for {len(df)} tickers is ready for your analysis.\n \n       \U0001F5FA You can explore the available symbols either in the CSV or by clicking the button below \U0001F447\n \n                      \U0001F680 Go ahead and pick the stocks you're interested in!")

            # Button to view available symbols
            show_button = widgets.Button(
                description="Show available symbols",
                button_style='info',
                layout=widgets.Layout(width='180px', height='30px', margin='20px 0 20px 0'),
                style={'font_weight': 'bold'}
            )

            # State for switching show/hide
            symbols_visible = {'state': False}

            def on_show_clicked(btn):
                with symbols_output:
                    if symbols_visible['state']:
                        symbols_output.clear_output()
                        symbols_visible['state'] = False
                        show_button.description = "Show available symbols"
                    else:
                        symbols_output.clear_output()
                        print("Symbols available for your analysis:\n")
                        row_size = 20
                        symbols_list = df.index.tolist()
                        for i in range(0, len(symbols_list), row_size):
                            print(", ".join(symbols_list[i:i+row_size]))
                        symbols_visible['state'] = True
                        show_button.description = "Hide symbols"

            show_button.on_click(on_show_clicked)

            toggle_ui = widgets.VBox(
                [show_button, symbols_output],
                layout=widgets.Layout(align_items='center')
            )

            display(toggle_ui)

    button.on_click(on_button_clicked)
    
    # Alignment
    ui = widgets.VBox(
        [dropdown, button, output],
        layout=widgets.Layout(
            align_items='center'
        )
    )

    display(ui)
    
    return result
    